In [1]:
import numpy as np
import profile

In [88]:
import decision_tree_helper
from decision_tree_helper import build_tree

class DecisionTree(object):
    """
    DecisionTree class, that represents one Decision Tree

    :param max_tree_depth: maximum depth for this tree.
    """
    def __init__(self, max_tree_depth):
        self.max_depth = max_tree_depth

    def fit(self, X, Y):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :param Y: 1 dimensional python list or numpy 1 dimensional array
        """
        data = X.tolist()
        for index in range(len(X)):
            data[index].append(Y[index])
        self.tree = build_tree(data, max_depth = self.max_depth)

    def predict(self, X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :return: Y - 1 dimension python list with labels
        """
        X = X.tolist()
        Y = []
        for row in X:
            current_node = self.tree     
            while(current_node != None and not current_node.is_leaf):
                feature_val = row[current_node.column] 
                if type(feature_val) == int or type(feature_val) == float:
                    if feature_val >= current_node.value:
                        current_node = current_node.true_branch
                    else: 
                        current_node = current_node.false_branch
                if type(feature_val) == str:
                    if feature_val == current_node.value:
                        current_node = current_node.true_branch
                    else:
                        current_node = current_node.false_branch
            Y.append(current_node.result)
        return Y


In [10]:
from decision_tree import DecisionTree

class RandomForest(object):
    """
    RandomForest a class, that represents Random Forests.

    :param num_trees: Number of trees in the random forest
    :param max_tree_depth: maximum depth for each of the trees in the forest.
    :param ratio_per_tree: ratio of points to use to train each of
        the trees.
    """
    def __init__(self, num_trees, max_tree_depth, ratio_per_tree=0.5):
        self.num_trees = num_trees
        self.max_tree_depth = max_tree_depth
        self.trees = None

    def fit(self, X, Y):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :param Y: 1 dimensional python list or numpy 1 dimensional array
        """
        self.trees = []
        ind = np.arange(X.shape[0])
        for _ in rang(num_trees):
            train_ind = np.random.choice(ind, int(X.shape[0]*ratio_per_tree), replace=False)
            tree_clf = DecisionTree(max_tree_depth)
            tree_clf.fit(X[train_ind], Y[train_ind])
            self.trees.append(tree_clf.tree)
        

    def predict(self, X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :return: (Y, conf), tuple with Y being 1 dimension python
        list with labels, and conf being 1 dimensional list with
        confidences for each of the labels.
        """
        Y_list = []
        for tree in self.trees:
            Y_list.append(tree.predict(X))
        Y = []
        conf = []
        for i in range(self.num_trees):
            Y_i_counter = get_columns(Y_list, i)
            y = max(Y_i_counter, key=lambda key: dict_current_results[key])
            Y.append(y)
            conf.append(Y_i_counter[y]/self.num_trees)
        return (Y, conf)
    
    
def get_columns(list_2D, cols):
    if type(cols) == int or type(cols) == float:
        return [item[cols] for item in list_2D]
    return [[item[col] for col in cols] for item in list_2D]




In [20]:
import numpy as np
import matplotlib.pyplot as plt
from logistic_regression import LogisticRegression
from decision_tree import DecisionTree

def accuracy_score(Y_true, Y_predict):
    tp_plus_tn = 0
    N = len(Y_true)
    for index in range(N):
        if Y_true[index] == Y_predict[index]:
            tp_plus_tn += 1
    return tp_plus_tn / N


def evaluate_performance():
    '''
    Evaluate the performance of decision trees and logistic regression,
    average over 1,000 trials of 10-fold cross validation

    Return:
      a matrix giving the performance that will contain the following entries:
      stats[0,0] = mean accuracy of decision tree
      stats[0,1] = std deviation of decision tree accuracy
      stats[1,0] = mean accuracy of logistic regression
      stats[1,1] = std deviation of logistic regression accuracy

    ** Note that your implementation must follow this API**
    '''

    # Load Data
    filename = 'data/SPECTF.dat'
    data = np.loadtxt(filename, delimiter=',')
    X = data[:, 1:]
    y = np.array(data[:, 0])
    n, d = X.shape

    all_accuracies_dt = []
    all_accuracies_lr = []
    for trial in range(1):
        idx = np.arange(n)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]
        
        ind = np.arange(X.shape[0])
        #classifier_dt = tree.DecisionTreeClassifier(max_depth=50)
        classifier_lr = LogisticRegression(max_steps=10000, epsilon=1e-7)
        scores_dt = []
        scores_lr = []
        for i in range(10):
            test_ind = np.random.choice(ind, int(X.shape[0]/10), replace=False)
            ind = np.setdiff1d(np.arange(X.shape[0]), test_ind)
            X_train, Y_train, X_test, Y_test = X[ind], y[ind], X[test_ind], y[test_ind]
            # train the decision tree
            classifier_dt.fit(X_train, Y_train)
            accuracy_dt = accuracy_score(Y_true=Y_test, Y_predict=classifier_dt.predict(X_test))
            scores_dt.append(accuracy_dt)
            classifier_lr.fit(X_train, Y_train)
            accuracy_lr = accuracy_score(Y_true=Y_test, Y_predict=classifier_lr.predict(X_test))
            scores_lr.append(accuracy_lr)
        all_accuracies_dt.append(np.mean(scores_dt))
        all_accuracies_lr.append(np.mean(scores_lr))
       


    # compute the training accuracy of the model
    meanDecisionTreeAccuracy = np.mean(all_accuracies_dt)
    stddevDecisionTreeAccuracy = np.std(all_accuracies_dt)
    # TODO: update these statistics based on the results of your experiment
    meanLogisticRegressionAccuracy = np.mean(all_accuracies_lr)
    stddevLogisticRegressionAccuracy = np.std(all_accuracies_lr)
    meanRandomForestAccuracy = 0
    stddevRandomForestAccuracy = 0

    # make certain that the return value matches the API specification
    stats = np.zeros((3, 2))
    stats[0, 0] = meanDecisionTreeAccuracy
    stats[0, 1] = stddevDecisionTreeAccuracy
    stats[1, 0] = meanRandomForestAccuracy
    stats[1, 1] = stddevRandomForestAccuracy
    stats[2, 0] = meanLogisticRegressionAccuracy
    stats[2, 1] = stddevLogisticRegressionAccuracy
    return stats


# Do not modify from HERE...
if __name__ == "__main__":
    stats = evaluate_performance()
    print("Decision Tree Accuracy = ", stats[0, 0], " (", stats[0, 1], ")")
    print("Random Forest Tree Accuracy = ", stats[1, 0], " (", stats[1, 1], ")")
    print("Logistic Reg. Accuracy = ", stats[2, 0], " (", stats[2, 1], ")")
# ...to HERE.


iteration= 0, and cost_value= 1.4505570899066784
the gradient descent algorithm is finished.
iteration= 0, and cost_value= 1.4505570899066784
the gradient descent algorithm is finished.
iteration= 0, and cost_value= 1.4555079080832756
the gradient descent algorithm is finished.
iteration= 0, and cost_value= 1.445606271730081
the gradient descent algorithm is finished.
iteration= 0, and cost_value= 1.4357046353768863
the gradient descent algorithm is finished.
iteration= 0, and cost_value= 1.4357046353768865
the gradient descent algorithm is finished.
iteration= 0, and cost_value= 1.4357046353768863
the gradient descent algorithm is finished.
iteration= 0, and cost_value= 1.4357046353768865
the gradient descent algorithm is finished.
iteration= 0, and cost_value= 1.445606271730081
the gradient descent algorithm is finished.
iteration= 0, and cost_value= 1.46540954443647
the gradient descent algorithm is finished.
Decision Tree Accuracy =  0.742307692308  ( 0.0 )
Random Forest Tree Accur

In [9]:
import numpy as np

class LogisticRegression(object):
    
    def __init__(self, epsilon=0.0001, l=1, step_size=0.01, max_steps=1000, initial_beta=None):
        self.epsilon = epsilon
        self.l = l
        self.step_size = step_size
        self.max_steps = max_steps
        self.initial_beta = initial_beta

    def fit(self, X, Y):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :param Y: 1 dimensional python list or numpy 1 dimensional array
        """
        self.beta = stochastic_gradient_descent(X = X, 
                                                Y = Y, 
                                                epsilon=self.epsilon, 
                                                l=self.l, 
                                                step_size=self.step_size, 
                                                max_steps=self.max_steps,
                                               initial_beta=self.initial_beta)

    def predict(self, X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :return: Y - 1 dimension python list with labels
        """
        Y = []
        for row in X:
            Y.append(int(np.dot(row, beta) >= 0))
        return Y

def sigmoid(s):
    return 1/(1+np.exp(-s))

def cost_function(X, Y, beta):
    h = sigmoid(np.dot(X, beta))
    return (-np.dot(Y, np.log(h))+ np.dot((1+Y),1-h))/X.shape[0]

def stochastic_gradient_descent(X, Y, epsilon=0.0001, l=1, step_size=0.01, max_steps=1000, initial_beta=None):
    """
    Implement gradient descent using stochastic approximation of the gradient.
    :param X: data matrix (2 dimensional np.array)
    :param Y: response variables (1 dimensional np.array)
    :param l: regularization parameter lambda
    :param epsilon: approximation strength
    :param max_steps: maximum number of iterations before algorithm will
        terminate.
    :return: value of beta (1 dimensional np.array)
    """
    N, D = X.shape[0], X.shape[1]
    X, l_vector, var_cols, std_cols, mean_cols = parameters_for_scaling(X, l)
    l_vector[0] = 0
    if initial_beta == None:
        beta = np.zeros(D)
    else:
        beta = initial_beta
    for s in range(max_steps):
        if s % N == 0:
            X, Y = shuffle_data(X, Y)
        next_beta = beta - step_size*normalized_gradient(X[s%N], Y[s%N], beta, l_vector)
        if s % 1000 == 0:
            print('iteration= {}, and cost_value= {}'.format(s, cost_function(X, Y, beta)))
        if np.linalg.norm(next_beta - beta)/np.linalg.norm(next_beta) < epsilon:
            print('the gradient descent algorithm is finished.')
            return get_real_beta(next_beta, std_cols, mean_cols)
        beta = next_beta
    print('the gradient descent algorithm is finished.')
    return get_real_beta(beta, std_cols, mean_cols)



def normalized_gradient(X, Y, beta, l):
    """
    :param X: data matrix (2 dimensional np.array)
    :param Y: response variables (1 dimensional np.array)
    :param beta: value of beta (1 dimensional np.array)
    :param l: regularization parameter lambda
    :return: normalized gradient, i.e. gradient normalized according to data
    """
    return (np.dot(X.T, sigmoid(np.dot(X,beta))-Y) + l*beta/2)/X.shape[0]

def parameters_for_scaling(X, l):
    X = np.copy(X)
    featchures_mat = X[:, 1:]
    var_cols = np.var(X, axis=0)
    std_cols = np.std(X, axis=0)
    mean_cols = np.mean(X, axis=0)
    var_cols[0] = 1
    featchures_mat = X[:, 1:]
    X[:, 1:] = div0(featchures_mat - np.mean(featchures_mat, axis=0),np.std(featchures_mat, axis=0))
    return X, div0(l,var_cols), var_cols, std_cols, mean_cols

def shuffle_data(X, Y):
    data = np.hstack((X, np.array(Y).reshape(len(Y), 1)))
    np.random.shuffle(data) 
    return data[:,:X.shape[1]], data[:,X.shape[1]:].reshape(len(Y))

def div0( a, b ):
    """ ignore / 0, div0( [-1, 0, 1], 0 ) -> [0, 0, 0] """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide( a, b )
        c[ ~ np.isfinite( c )] = 0  # -inf inf NaN
    return c

In [4]:
from decision_tree import DecisionTree
filename = 'data/SPECTF.dat'
data = np.loadtxt(filename, delimiter=',')
data.shape
X = data[:, 1:]
y = np.array(data[:, 0])
clf_dt = DecisionTree(50)
profile.run('print(clf_dt.fit(X,y)); print()')

None

         509661 function calls (509605 primitive calls) in 3.172 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 :0(acquire)
    21324    0.078    0.000    0.078    0.000 :0(append)
    38418    0.141    0.000    0.141    0.000 :0(array)
    38418    0.125    0.000    0.125    0.000 :0(dot)
        1    0.000    0.000    3.172    3.172 :0(exec)
        3    0.000    0.000    0.000    0.000 :0(getpid)
        3    0.000    0.000    0.000    0.000 :0(isinstance)
   191661    0.297    0.000    0.297    0.000 :0(len)
       57    0.000    0.000    0.000    0.000 :0(max)
     1260    0.047    0.000    0.078    0.000 :0(min)
        2    0.000    0.000    0.000    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000    0.000    0.000 :0(tolist)
        4    0.000    0.000    0.000    0.000 :0(urandom)
        1    0.000    0.00

In [3]:
from decision_tree import DecisionTree
filename = 'data/SPECTF.dat'
data = np.loadtxt(filename, delimiter=',')
data.shape
X = data[:, 1:]
y = np.array(data[:, 0])
clf_dt = DecisionTree(50)
profile.run('print(clf_dt.fit(X,y)); print()')

None

         509661 function calls (509605 primitive calls) in 2.609 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 :0(acquire)
    21324    0.078    0.000    0.078    0.000 :0(append)
    38418    0.078    0.000    0.078    0.000 :0(array)
    38418    0.125    0.000    0.125    0.000 :0(dot)
        1    0.000    0.000    2.609    2.609 :0(exec)
        3    0.000    0.000    0.000    0.000 :0(getpid)
        3    0.000    0.000    0.000    0.000 :0(isinstance)
   191661    0.328    0.000    0.328    0.000 :0(len)
       57    0.000    0.000    0.000    0.000 :0(max)
     1260    0.062    0.000    0.078    0.000 :0(min)
        2    0.000    0.000    0.000    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000    0.000    0.000 :0(tolist)
        4    0.000    0.000    0.000    0.000 :0(urandom)
        1    0.000    0.00

In [18]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth=50)
profile.run('print(clf.fit(X,y)); print()')

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=50,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

         1067 function calls (940 primitive calls) in 0.016 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       20    0.000    0.000    0.000    0.000 :0(__subclasses__)
       36    0.000    0.000    0.000    0.000 :0(_filters_mutated)
        4    0.000    0.000    0.000    0.000 :0(_getframe)
        4    0.000    0.000    0.000    0.000 :0(acquire)
       28    0.000    0.000    0.000    0.000 :0(add)
        1    0.000    0.000    0.000    0.000 :0(any)
       40    0.000    0.000    0.000    0.000 :0(append)
        1    0.000    0.000    0.000    0.000 :0(argsort)
       15    0.000    0.000    0.000    0.000

In [31]:
test_arr = np.array([[1.2, 'yes'], [5, 'no'], [7.8, 'yes']])
np.dot(test_arr[:, 0].astype(float), test_arr[:, 0].astype(float))